# Tutorial 3: Bus Engine Replacement with Unobserved Heterogeneity

This notebook implements the bus engine replacement model from:

**"Conditional Choice Probability Estimation of Dynamic Discrete Choice Models With Unobserved Heterogeneity"**  
*Arcidiacono & Miller, Econometrica (2011)*

Building on the classic Rust (1987) framework.

## Overview

### Part 1: Generic Framework
- State enumeration and indexing
- Fixed-point iteration
- Abstract interface for dynamic models

### Part 2: Bus Engine Model
- State space: mileage × mileage-increment parameter
- Transition: exponential mileage increments
- Finite dependence: replacement resets mileage
- Value function iteration and simulation

### Part 3: Estimation Without Heterogeneity
- **NFXP**: Nested fixed point (Rust 1987)
- **Two-Step CCP**: Hotz-Miller inversion

### Part 4: Estimation With Unobserved Heterogeneity
- Why Two-Step fails with unknown types
- **NFXP + EM**: Full solution with expectation-maximization
- **CCP-EM (Data)**: Update CCPs via weighted logit
- **CCP-EM (Model)**: Update CCPs via model operator

> **Note on Grid Resolution**: This notebook uses a coarse grid (x_step=0.5, z_step=0.05) for fast execution.
> For research applications, use the fine grid (x_step=0.125, z_step=0.01) matching the original MATLAB code.
> The coarse grid may cause parameter bias but correctly demonstrates the estimation methods.

---
## 0.1 Setup

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import expon
from dataclasses import dataclass, field
from typing import Tuple, Dict, List, Optional, Callable, Any
from abc import ABC, abstractmethod
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

EULER_CONSTANT = 0.5772156649015329

### 0.2 Logit Utilities

In [ ]:
def logit_prob(v: np.ndarray) -> np.ndarray:
    """
    Binary logit: convert value difference to choice probability.
    
    P(d=1) = exp(v) / (1 + exp(v)) where v = v_1 - v_0
    """
    v = np.asarray(v, dtype=float)
    result = np.zeros_like(v)
    pos = v >= 0
    result[pos] = 1.0 / (1.0 + np.exp(-v[pos]))
    result[~pos] = np.exp(v[~pos]) / (1.0 + np.exp(v[~pos]))
    return result


def log_odds(p: np.ndarray) -> np.ndarray:
    """Inverse logit (Hotz-Miller inversion): p → v_1 - v_0"""
    p = np.clip(p, 1e-10, 1 - 1e-10)
    return np.log(p / (1 - p))


def emax_logit(v: np.ndarray) -> np.ndarray:
    """
    Expected maximum of logit: E[max(v_0 + ε_0, v_1 + ε_1)]
    
    With v_0 = 0: log(1 + exp(v_1)) + γ
    """
    # Numerically stable log-sum-exp
    v = np.asarray(v, dtype=float)
    max_v = np.maximum(0, v)
    return max_v + np.log(np.exp(-max_v) + np.exp(v - max_v)) + EULER_CONSTANT


def emax_from_ccp(p: np.ndarray) -> np.ndarray:
    """
    Ex-ante value from CCPs (Hotz-Miller).
    
    V(x) = γ - log(1 - p) where p = P(d=1|x)
    (Assumes v_0 = 0 normalization)
    """
    p = np.clip(p, 1e-10, 1 - 1e-10)
    return EULER_CONSTANT - np.log(1 - p)


# Quick tests
print("Logit utilities:")
print(f"  v=0 → p={logit_prob(0):.3f} (should be 0.5)")
print(f"  v=2 → p={logit_prob(2):.3f}")
print(f"  p=0.5 → V={emax_from_ccp(0.5):.3f} (should be γ + log(2) ≈ 1.27)")

---
# Part 1: Generic Framework

## Class Hierarchy

```
StateSpace              → State enumeration with index mapping
FixedPointSolver        → Solves x* = T(x*) via iteration

DynamicModel (ABC)      → Interface for any DDC model
    ├── state_space, n_actions, discount
    ├── flow_utility(x, d)
    └── ccp_operator(ccps), solve()

BusEngineModel          → Concrete implementation (Rust 1987)
```

---
## 1.1 StateSpace

In [ ]:
class StateSpace:
    """
    Maps states ↔ integer indices for array storage.
    
    For bus engine: state = (x_bin, z_bin) where
    - x_bin: mileage bin index
    - z_bin: mileage increment parameter bin index
    """
    
    def __init__(self, states: List[Any]):
        self._states = list(states)
        self._index_map = {s: i for i, s in enumerate(self._states)}
    
    @property
    def n_states(self) -> int:
        return len(self._states)
    
    def state_to_index(self, state: Any) -> int:
        return self._index_map[state]
    
    def index_to_state(self, idx: int) -> Any:
        return self._states[idx]
    
    def states(self) -> List[Any]:
        return self._states.copy()
    
    def __len__(self) -> int:
        return len(self._states)
    
    def __repr__(self) -> str:
        return f"StateSpace(n_states={self.n_states})"

---
## 1.2 FixedPointSolver

In [ ]:
@dataclass
class FixedPointResult:
    """Results from fixed-point iteration."""
    solution: np.ndarray
    n_iterations: int
    converged: bool
    final_diff: float
    computation_time: float


class FixedPointSolver:
    """Solve x* = T(x*) by iteration."""
    
    def __init__(self, 
                 operator: Callable[[np.ndarray], np.ndarray],
                 tolerance: float = 1e-8,
                 max_iter: int = 1000,
                 verbose: bool = True):
        self.operator = operator
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.verbose = verbose
    
    def solve(self, initial: np.ndarray) -> FixedPointResult:
        start = time.time()
        x = initial.copy()
        
        for iteration in range(self.max_iter):
            x_new = self.operator(x)
            diff = np.max(np.abs(x_new - x))
            
            if self.verbose and (iteration + 1) % 100 == 0:
                print(f"  Iter {iteration+1}: diff = {diff:.2e}")
            
            if diff < self.tolerance:
                if self.verbose:
                    print(f"  Converged in {iteration+1} iterations")
                return FixedPointResult(x_new, iteration+1, True, diff, time.time()-start)
            
            x = x_new
        
        if self.verbose:
            print(f"  Did not converge after {self.max_iter} iterations")
        return FixedPointResult(x, self.max_iter, False, diff, time.time()-start)


class DynamicModel(ABC):
    """
    Abstract base class for dynamic discrete choice models.
    
    Same interface as Tutorial 2 - ensures consistent structure across tutorials.
    """
    
    @property
    @abstractmethod
    def state_space(self) -> StateSpace:
        """The state space."""
        pass
    
    @property
    @abstractmethod
    def n_actions(self) -> int:
        """Number of actions."""
        pass
    
    @property
    @abstractmethod
    def discount(self) -> float:
        """Discount factor β."""
        pass
    
    @abstractmethod
    def flow_utility(self, state: Any, action: int) -> float:
        """Flow utility u(x, d)."""
        pass
    
    @abstractmethod
    def transition_probs(self, state: Any, action: int) -> Dict[Any, float]:
        """Transition probabilities P(x' | x, d)."""
        pass
    
    @abstractmethod
    def ccp_operator(self, ccps: np.ndarray) -> np.ndarray:
        """CCP operator: maps CCPs to best-response CCPs."""
        pass
    
    def solve(self, 
              initial_ccps: np.ndarray = None,
              tolerance: float = 1e-8,
              max_iter: int = 1000,
              verbose: bool = True) -> FixedPointResult:
        """
        Solve for optimal/equilibrium CCPs using FixedPointSolver.
        
        This is the same pattern used in Tutorial 2.
        """
        if initial_ccps is None:
            initial_ccps = 0.5 * np.ones(self.state_space.n_states)
        solver = FixedPointSolver(self.ccp_operator, tolerance, max_iter, verbose)
        return solver.solve(initial_ccps)

---
# Part 2: Bus Engine Model

## The Rust (1987) Framework

**Decision**: Each period, Harold Zurcher decides whether to:
- $d=0$: Replace engine (reset mileage)
- $d=1$: Keep engine (mileage accumulates)

**State variables**:
- $x$: Accumulated mileage (discretized)
- $z$: Mileage increment parameter (bus-specific, time-invariant)
- $s \in \{0, 1\}$: Unobserved type (for heterogeneity extension)

**Flow utility**:
$$u(x, d, s) = \begin{cases} 
0 & \text{if } d = 0 \text{ (replace, normalized)} \\
\alpha_1 + \alpha_2 x + \alpha_3 s & \text{if } d = 1 \text{ (keep)}
\end{cases}$$

**Transition** (mileage increment $\Delta x \sim \text{Exp}(z)$):
$$x' = \begin{cases}
\Delta x & \text{if } d = 0 \text{ (reset)} \\
x + \Delta x & \text{if } d = 1
\end{cases}$$

**Finite dependence**: Replacement resets mileage → value difference simplifies.

> **Convention note:** We use $d=1$ for "keep" and $d=0$ for "replace." This differs from AM2011's $d_1$=replace, $d_2$=keep notation, but the economics is identical.

## 2.1 Parameters

In [ ]:
@dataclass
class BusEngineParams:
    """
    Parameters for bus engine replacement model.
    
    Matches MATLAB busprograms structure.
    
    Flow Utility (for keep d=1)
    ----------------------------------
    u(x, s) = intercept + mileage_coef * x + type_coef * s
    
    where:
    - x: mileage (in units of 10,000 miles)
    - s: unobserved type (0 or 1)
    
    Replace (d=0) has utility normalized to 0.
    """
    # Structural parameters
    intercept: float = 2.0        # α_1: base replacement value
    mileage_coef: float = -0.15   # α_2: mileage effect (negative → replace when high)
    type_coef: float = 1.0        # α_3: type effect on replacement
    discount: float = 0.9         # β: discount factor
    type_prob: float = 0.4        # π: P(type = 0)
    
    # Initial values for estimation (can be modified for experiments)
    theta0_intercept: float = 2.0     # Initial guess for α_1
    theta0_mileage: float = -0.1      # Initial guess for α_2
    theta0_type: float = 0.5          # Initial guess for α_3
    
    # State space discretization
    x_min: float = 0.0             # Min mileage
    x_max: float = 25.0          # Max mileage (25 = 250,000 miles)
    x_step: float = 0.125        # May use coarser grid for demo (use 0.125 for full resolution)         # Mileage bin width
    z_min: float = 0.25           # Min mileage increment rate
    z_max: float = 1.25           # Max mileage increment rate
    z_step: float = 0.01          # May use coarser grid for demo (use 0.01 for full resolution)          # Z bin width
    
    @property
    def x_grid(self) -> np.ndarray:
        """Mileage grid points."""
        return np.arange(self.x_min, self.x_max + self.x_step/2, self.x_step)
    
    @property
    def z_grid(self) -> np.ndarray:
        """Z parameter grid points."""
        return np.arange(self.z_min, self.z_max + self.z_step/2, self.z_step)
    
    @property
    def n_x(self) -> int:
        return len(self.x_grid)
    
    @property
    def n_z(self) -> int:
        return len(self.z_grid)
    
    def theta0_vector(self, fix_discount: bool = True) -> np.ndarray:
        """Initial parameter guess for estimation."""
        if fix_discount:
            return np.array([self.theta0_intercept, self.theta0_mileage, self.theta0_type])
        return np.array([self.theta0_intercept, self.theta0_mileage, self.theta0_type, self.discount])
    
    def theta_vector(self) -> np.ndarray:
        """Structural parameters as vector [α_1, α_2, α_3, β]."""
        return np.array([self.intercept, self.mileage_coef, self.type_coef, self.discount])
    
    @classmethod
    def from_vector(cls, theta: np.ndarray, **kwargs):
        """Create from parameter vector."""
        return cls(
            intercept=theta[0],
            mileage_coef=theta[1],
            type_coef=theta[2] if len(theta) > 2 else 0.0,
            discount=theta[3] if len(theta) > 3 else 0.9,
            **kwargs
        )


# Display default parameters
from tabulate import tabulate

params = BusEngineParams()
param_table = [
    ["Structural (α)", str(params.theta_vector())],
    ["Type probability (π)", params.type_prob],
    ["Mileage grid", f"{params.n_x} bins from {params.x_min} to {params.x_max}"],
    ["Z grid", f"{params.n_z} bins from {params.z_min} to {params.z_max}"],
    ["Total states per type", params.n_x * params.n_z]
]
print("Bus Engine Parameters:\n")
print(tabulate(param_table, headers=["Parameter", "Value"], tablefmt="simple"))

## 2.2 Transition Matrices

Mileage increment $\Delta x \sim \text{Exp}(z)$ with rate parameter $z$.

Discretized transition:
$$P(x' | x, z, d=0) = F(x_{j+1} - x) - F(x_j - x)$$

where $F(\cdot)$ is the exponential CDF.

In [ ]:
def build_transition_matrix(z: float, x_grid: np.ndarray) -> np.ndarray:
    """
    Build mileage transition matrix for given z.
    
    Parameters
    ----------
    z : float
        Exponential rate parameter
    x_grid : array
        Mileage grid points
    
    Returns
    -------
    P : array, shape (n_x, n_x)
        P[i, j] = P(x' = x_grid[j] | x = x_grid[i], d=0)
    """
    n = len(x_grid)
    x_ub = np.append(x_grid[1:], np.inf)  # Upper bounds for bins
    
    P = np.zeros((n, n))
    for i in range(n):
        # CDF of increment reaching each bin
        cdf = 1 - np.exp(-z * (x_ub - x_grid[i]))
        cdf = np.clip(cdf, 0, 1)
        
        # Probability mass in each bin
        P[i, :] = np.diff(np.insert(cdf, 0, 0))
        
        # Only transitions to higher states (or stay if at max)
        P[i, :i] = 0
        P[i, :] = P[i, :] / P[i, :].sum()  # Renormalize
    
    return P


In [ ]:
# Example: transition matrix for z = 0.5
x_test = np.arange(0, 5, 0.5)
P_test = build_transition_matrix(0.5, x_test)
param_table = [
    ["Shape", f"{P_test.shape}"],
    ["Row sums", f"{P_test.sum(axis=1)}"],
    ["P[0,:5] (from x=0)", f"{P_test[0, :7].round(3)}"],
    ["P[3,:7] (from x=1.5)", f"{P_test[3, :7].round(3)}"],
]

print("Transition matrix (z=0.5, small grid):\n")
print(tabulate(param_table, headers=["Type", "Value"], tablefmt="simple"))


## 2.3 BusEngineModel Class

In [ ]:
class BusEngineModel(DynamicModel):
    """
    Bus engine replacement model (Rust 1987).
    
    Extends DynamicModel interface from Tutorial 2.
    
    State: (x_idx, z_idx) - mileage bin and z-parameter bin
    Action: d ∈ {0, 1} - replace (0) or keep (1)
    
    For heterogeneity, we solve separately for each type s.
    """
    
    def __init__(self, params: BusEngineParams = None, type_s: int = 0):
        self.params = params or BusEngineParams()
        self.type_s = type_s  # Type for this instance
        self._build_state_space()
        self._build_transitions()
    
    def _build_state_space(self):
        """Build state space: (x_idx, z_idx) pairs."""
        p = self.params
        states = [
            (x_idx, z_idx)
            for z_idx in range(p.n_z)
            for x_idx in range(p.n_x)
        ]
        self._state_space = StateSpace(states)
        
        # Precompute x and z values for each state index
        self._x_vals = np.array([p.x_grid[s[0]] for s in states])
        self._z_vals = np.array([p.z_grid[s[1]] for s in states])
        self._x_idx = np.array([s[0] for s in states])
        self._z_idx = np.array([s[1] for s in states])
    
    def _build_transitions(self):
        """
        Build transition matrices for each z value.
        
        Stores:
        - P_continue[z_idx]: transition if d=1 (keep, mileage accumulates)
        - P_replace[z_idx]: transition if d=0 (replace, reset to x=0)
        """
        p = self.params
        self._P_continue = []
        self._P_replace = []
        
        for z_idx, z in enumerate(p.z_grid):
            P = build_transition_matrix(z, p.x_grid)
            self._P_continue.append(P)
            self._P_replace.append(P[0, :])  # First row = transitions from x=0
    
    # =========================================================================
    # DynamicModel Interface (from Tutorial 2)
    # =========================================================================
    
    @property
    def state_space(self) -> StateSpace:
        return self._state_space
    
    @property
    def n_actions(self) -> int:
        return 2  # Continue or Replace
    
    @property
    def discount(self) -> float:
        return self.params.discount
    
    def flow_utility(self, state: Tuple[int, int], action: int) -> float:
        """
        Flow utility u(x, d, s).
        
        Replace (d=0): u_0 = 0 (normalized)
        Keep (d=1):    u_1 = α_1 + α_2 * x + α_3 * s
        """
        if action == 0:
            return 0.0
        
        x_idx, z_idx = state
        p = self.params
        x = p.x_grid[x_idx]
        # MATLAB: util1 = alpha(1) + alpha(2)*xval(x) + alpha(3)*s
        # Uses raw x value (0 to 25), not x/10
        return p.intercept + p.mileage_coef * x + p.type_coef * self.type_s
    
    def transition_probs(self, state: Tuple[int, int], action: int) -> Dict[Tuple[int, int], float]:
        """
        Transition probabilities P(x' | x, d).
        
        Continue: mileage increases stochastically
        Replace: mileage resets to 0, then increases
        """
        x_idx, z_idx = state
        p = self.params
        
        if action == 0:  # Replace (d=0)
            P_row = self._P_replace[z_idx]
        else:  # Keep (d=1)
            P_row = self._P_continue[z_idx][x_idx, :]
        
        return {(x_new, z_idx): P_row[x_new] for x_new in range(p.n_x) if P_row[x_new] > 1e-10}
    
    def ccp_operator(self, ccps: np.ndarray) -> np.ndarray:
        """
        CCP operator: Ψ(p) maps current CCPs to best-response CCPs.
        
        This is the KEY method that FixedPointSolver uses.
        
        For single-agent:
        1. Compute value V(x) from CCPs via Hotz-Miller inversion
        2. Compute expected continuation values
        3. Compute value difference v_1 - v_0
        4. Apply logit to get new CCPs
        """
        p = self.params
        n_x, n_z = p.n_x, p.n_z
        
        ccps = ccps.reshape(n_z, n_x)
        ccps_new = np.zeros_like(ccps)
        
        # Flow utility for keep (d=1) (vectorized over x)
        # MATLAB uses raw x values
        u1 = p.intercept + p.mileage_coef * p.x_grid + p.type_coef * self.type_s
        
        for z_idx in range(n_z):
            # Hotz-Miller: W(x) = γ - log(P(replace|x))
            p_replace = np.clip(1 - ccps[z_idx, :], 1e-10, 1 - 1e-10)
            V = EULER_CONSTANT - np.log(p_replace)
            
            # Expected continuation values
            P_cont = self._P_continue[z_idx]
            P_repl = self._P_replace[z_idx]

            EV_continue = P_cont @ V      # E[W' | d=1, keep]
            EV_replace = P_repl @ V       # E[W' | d=0, replace] (same for all x since reset)
            
            # Value difference: v_1 - v_0
            v_diff = u1 + p.discount * (EV_continue - EV_replace)
            
            # New CCPs via logit
            ccps_new[z_idx, :] = logit_prob(v_diff)
        
        return ccps_new.flatten()
    
    def solve(self, 
              initial_ccps: np.ndarray = None,
              tolerance: float = 1e-8,
              max_iter: int = 1000,
              verbose: bool = True) -> FixedPointResult:
        """
        Solve for optimal CCPs using FixedPointSolver.
        
        Finds p* such that p* = Ψ(p*).
        
        This is inherited pattern from Tutorial 2's DynamicModel.
        """
        if initial_ccps is None:
            initial_ccps = 0.1 * np.ones(self.state_space.n_states)
        
        solver = FixedPointSolver(self.ccp_operator, tolerance, max_iter, verbose)
        return solver.solve(initial_ccps)


In [ ]:
# Test model construction
model = BusEngineModel()
param_table = [
    ["States", f"{model.state_space.n_states}"],
    ["State space", f"{model.state_space}"],
]

print("="*70)
print("BusEngineModel")
print("="*70)

print(tabulate(param_table, headers=["Type", "Value"], tablefmt="simple"))

## 2.4 Solve and Visualize

In [ ]:
# Solve for optimal CCPs using FixedPointSolver (same pattern as Tutorial 2)
print("="*70)
print("Solving for Optimal CCPs")
print("="*70)

# Create models for each type
model_type0 = BusEngineModel(type_s=0)
model_type1 = BusEngineModel(type_s=1)

# Solve using FixedPointSolver (via model.solve())
print("\nType 0:")
result_type0 = model_type0.solve(tolerance=1e-8, verbose=True)

print("\nType 1:")
result_type1 = model_type1.solve(tolerance=1e-8, verbose=True)

# Extract CCPs
ccps_type0 = result_type0.solution
ccps_type1 = result_type1.solution

print(f"\nBoth types converged: {result_type0.converged and result_type1.converged}")

In [ ]:
# Reshape for plotting
p = model_type0.params
ccps0 = ccps_type0.reshape(p.n_z, p.n_x)
ccps1 = ccps_type1.reshape(p.n_z, p.n_x)

# Plot replacement probability vs mileage for different z values
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Sample z values (evenly spaced)
z_indices = np.linspace(0, p.n_z - 1, 5, dtype=int)
colors = plt.cm.viridis(np.linspace(0, 1, len(z_indices)))

for ax, (ccps, title) in zip(axes, [(ccps0, 'Type s=0'), (ccps1, 'Type s=1')]):
    for i, z_idx in enumerate(z_indices):
        z_val = p.z_grid[z_idx]
        ax.plot(p.x_grid, ccps[z_idx, :], color=colors[i], label=f'z={z_val:.2f}')
    ax.set_xlabel('Mileage (x)')
    ax.set_ylabel('P(Keep)')
    ax.set_title(f'Keep Probability - {title}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2.5 Simulation

In [ ]:
@dataclass
class SimulationConfig:
    """
    Configuration for simulation.

    Matches MATLAB replication (hetero10ns/shellbusdatacomp420cond.m):
      N=2000, T2=10 usable periods, burn-in=10 (total forward sim = 20)
    """
    n_buses: int = 2000
    n_periods: int = 10
    burn_in: int = 10
    seed: int = 42

class Simulator:
    """
    Simulate panel data from the bus engine model.
    
    Same structure as Tutorial 2's Simulator class.
    
    Generates bus-level observations with state variables and choices.
    Each observation includes:
    - bus_id: Bus identifier
    - period: Time period
    - x, x_idx: Mileage and bin index
    - z, z_idx: Z parameter and bin index  
    - type_s: Unobserved type (0 or 1)
    - replace: Replacement decision (0 or 1)
    """
    
    def __init__(self, 
                 model_type0: BusEngineModel,
                 model_type1: BusEngineModel,
                 ccps_type0: np.ndarray,
                 ccps_type1: np.ndarray,
                 config: SimulationConfig):
        self.model0 = model_type0
        self.model1 = model_type1
        self.ccps0 = ccps_type0
        self.ccps1 = ccps_type1
        self.config = config
        self.params = model_type0.params
    
    def simulate(self) -> pd.DataFrame:
        """
        Simulate panel data.
        
        Returns
        -------
        pd.DataFrame
            Columns: bus_id, period, x, x_idx, z, z_idx, type_s, replace
        """
        np.random.seed(self.config.seed)
        p = self.params
        
        # Reshape CCPs for indexing
        ccps0 = self.ccps0.reshape(p.n_z, p.n_x)
        ccps1 = self.ccps1.reshape(p.n_z, p.n_x)
        
        # Draw types and z values for each bus
        types = (np.random.random(self.config.n_buses) > p.type_prob).astype(int)
        z_idx = np.random.randint(0, p.n_z, self.config.n_buses)
        
        # Initialize mileage at 0
        x_idx = np.zeros(self.config.n_buses, dtype=int)
        
        records = []
        total_periods = self.config.burn_in + self.config.n_periods
        
        for t in range(total_periods):
            # Get CCPs for each bus based on type
            ccps = np.where(
                types == 0,
                ccps0[z_idx, x_idx],
                ccps1[z_idx, x_idx]
            )
            
            # Draw replacement decisions
            replace = (np.random.random(self.config.n_buses) >= ccps).astype(int)  # replace when random >= P(keep)
            
            # Record observations (after burn-in)
            if t >= self.config.burn_in:
                for i in range(self.config.n_buses):
                    records.append({
                        'bus_id': i,
                        'period': t - self.config.burn_in,
                        'x': p.x_grid[x_idx[i]],
                        'x_idx': x_idx[i],
                        'z': p.z_grid[z_idx[i]],
                        'z_idx': z_idx[i],
                        'type_s': types[i],
                        'replace': replace[i]
                    })
            
            # Transition: draw next mileage
            for i in range(self.config.n_buses):
                if replace[i] == 1:
                    P_row = self.model0._P_replace[z_idx[i]]
                else:
                    P_row = self.model0._P_continue[z_idx[i]][x_idx[i], :]
                
                x_idx[i] = np.random.choice(p.n_x, p=P_row)
        
        return pd.DataFrame(records)


In [ ]:
config = SimulationConfig()
simulator = Simulator(model_type0, model_type1, ccps_type0, ccps_type1, config)
data = simulator.simulate()

In [ ]:
# Simulate data using Simulator class (same pattern as Tutorial 2)
print("="*70)
print("Simulating Data")
print("="*70)

# Configuration table
config_table = [
    ["n_buses", config.n_buses],
    ["n_periods", config.n_periods],
    ["burn_in", config.burn_in],
]
print("\nConfiguration:")
print(tabulate(config_table, headers=["Parameter", "Value"], tablefmt="simple"))

# Generated data summary
data_table = [
    ["Observations", len(data)],
    ["Buses", data['bus_id'].nunique()],
    ["Periods", data['period'].nunique()],
]
print("\nGenerated data:")
print(tabulate(data_table, headers=["Metric", "Value"], tablefmt="simple"))

# Type distribution and replacement rate
type_stats = data.groupby('type_s').agg(
    count=('bus_id', 'size'),
    replace_rate=('replace', 'mean')
).reset_index()
type_table = [[row['type_s'], row['count'], f"{row['replace_rate']:.4f}"]
              for _, row in type_stats.iterrows()]
print("\nType distribution and replacement rate:")
print(tabulate(type_table, headers=["Type", "Count", "Replace Rate"], tablefmt="simple"))

# Sample observations
print("\nSample observations:")
print(tabulate(data.head(10), headers='keys', tablefmt="simple", showindex=False))

# Keep reference to model for later use
model = model_type0

In [ ]:
# Visualize simulated data
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# 1. Replacement rate by mileage
ax = axes[0]
for s in [0, 1]:
    subset = data[data['type_s'] == s]
    bins = pd.cut(subset['x'], bins=20)
    rates = subset.groupby(bins)['replace'].mean()
    ax.plot(range(len(rates)), rates.values, 'o-', label=f'Type {s}', alpha=0.7)
ax.set_xlabel('Mileage bin')
ax.set_ylabel('Replacement rate')
ax.set_title('Replacement Rate by Mileage')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Mileage distribution
ax = axes[1]
for s in [0, 1]:
    subset = data[data['type_s'] == s]
    ax.hist(subset['x'], bins=30, alpha=0.5, label=f'Type {s}', density=True)
ax.set_xlabel('Mileage (x)')
ax.set_ylabel('Density')
ax.set_title('Mileage Distribution by Type')
ax.legend()

# 3. Replacement rate over time
ax = axes[2]
for s in [0, 1]:
    subset = data[data['type_s'] == s]
    rates = subset.groupby('period')['replace'].mean()
    ax.plot(rates.index, rates.values, 'o-', label=f'Type {s}', alpha=0.7)
ax.set_xlabel('Period')
ax.set_ylabel('Replacement rate')
ax.set_title('Replacement Rate Over Time')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
# Part 3: Estimation Without Heterogeneity

We first consider estimation when type $s$ is **observed** (or there is no heterogeneity).

## Methods

| Method | Solves DP? | Iteration? | Pros | Cons |
|--------|------------|------------|------|------|
| **NFXP** | Yes (every θ) | No | Consistent, efficient | Slow |
| **Two-Step CCP** | No | No | Very fast | Less efficient |

Both work when types are known. The key difference:
- NFXP: Solve Bellman equation at each parameter guess
- Two-Step: Estimate CCPs once, then linear regression

## 3.1 NFXP Estimator (Rust 1987)

**Algorithm**:
1. Given parameter guess θ, solve value function V(x; θ)
2. Compute choice probabilities P(d|x; θ)
3. Evaluate log-likelihood
4. Optimize over θ

$$\mathcal{L}(\theta) = \sum_{i,t} \log P(d_{it} | x_{it}; \theta)$$

In [ ]:
class NFXPEstimator:
    """
    Nested Fixed Point estimator (Rust 1987).
    
    For each parameter guess, solves the full dynamic programming problem
    using the model's solve() method (via FixedPointSolver).
    
    Estimates [α₁, α₂, α₃, logit(β)] matching MATLAB likebusML4.m.
    β is recovered via inverse logit: β = exp(logit_β) / (1 + exp(logit_β)).
    """
    
    def __init__(self, data: pd.DataFrame, base_params: BusEngineParams = None):
        self.data = data
        self.base_params = base_params or BusEngineParams()
        
        # Extract data arrays for fast computation
        self.x_idx = data['x_idx'].values
        self.z_idx = data['z_idx'].values
        self.replace = data['replace'].values
        self.n_obs = len(data)
        
        # If type is observed, use it; otherwise assume homogeneous
        if 'type_s' in data.columns:
            self.type_s = data['type_s'].values
        else:
            self.type_s = np.zeros(self.n_obs, dtype=int)
    
    @staticmethod
    def _logit_to_beta(logit_beta: float) -> float:
        """Inverse logit transform: β = exp(x)/(1+exp(x))."""
        return np.exp(logit_beta) / (1.0 + np.exp(logit_beta))
    
    @staticmethod
    def _beta_to_logit(beta: float) -> float:
        """Logit transform: logit(β) = log(β) - log(1-β)."""
        return np.log(beta) - np.log(1.0 - beta)
    
    def _theta_to_params(self, theta: np.ndarray) -> BusEngineParams:
        """
        Convert optimization vector [α₁, α₂, α₃, logit(β)] to BusEngineParams.
        Matches MATLAB likebusML4.m logit transform for β.
        """
        alpha = theta[:3]
        beta = self._logit_to_beta(theta[-1])
        return BusEngineParams(
            intercept=alpha[0],
            mileage_coef=alpha[1],
            type_coef=alpha[2],
            discount=beta,
        )
    
    def neg_log_likelihood(self, theta: np.ndarray) -> float:
        """
        Compute negative log-likelihood.
        
        theta = [α₁, α₂, α₃, logit(β)] where β = σ(logit_β).
        For each θ, creates models and solves for CCPs using FixedPointSolver.
        """
        params = self._theta_to_params(theta)
        
        # Create and solve for each type
        model0 = BusEngineModel(params, type_s=0)
        model1 = BusEngineModel(params, type_s=1)
        
        # Use solve() which internally uses FixedPointSolver
        result0 = model0.solve(tolerance=1e-6, max_iter=200, verbose=False)
        result1 = model1.solve(tolerance=1e-6, max_iter=200, verbose=False)
        
        ccps0 = result0.solution.reshape(params.n_z, params.n_x)
        ccps1 = result1.solution.reshape(params.n_z, params.n_x)
        
        # Get P(keep) for each observation (model computes P(d=1) = P(keep))
        p_keep = np.where(
            self.type_s == 0,
            ccps0[self.z_idx, self.x_idx],
            ccps1[self.z_idx, self.x_idx]
        )
        p_keep = np.clip(p_keep, 1e-10, 1 - 1e-10)
        
        # Log-likelihood: P(keep) when replace=0, P(replace)=1-P(keep) when replace=1
        ll = np.sum(
            (1 - self.replace) * np.log(p_keep) + 
            self.replace * np.log(1 - p_keep)
        )
        
        return -ll
    
    def estimate(self, theta0: np.ndarray = None) -> dict:
        """
        Estimate parameters [α₁, α₂, α₃, β] via MLE.
        
        Optimizes over [α₁, α₂, α₃, logit(β)] and converts back.
        Matches MATLAB likebusML4.m logit transform for β.
        """
        if theta0 is None:
            logit_beta0 = self._beta_to_logit(self.base_params.discount)
            theta0 = np.array([
                self.base_params.theta0_intercept,
                self.base_params.theta0_mileage,
                self.base_params.theta0_type,
                logit_beta0,
            ])
        
        start_time = time.time()
        
        result = minimize(self.neg_log_likelihood, theta0, method='Nelder-Mead',
                         options={'maxiter': 500, 'disp': True})
        
        # Convert back: replace logit(β) with β in the result vector
        theta_hat = result.x.copy()
        theta_hat[-1] = self._logit_to_beta(result.x[-1])
        
        return {
            'theta': theta_hat,
            'log_likelihood': -result.fun,
            'converged': result.success,
            'n_iterations': result.nit,
            'computation_time': time.time() - start_time
        }

In [ ]:
# Run NFXP estimation (with known types)
print("="*70)
print("NFXP Estimation (types observed)")
print("="*70)

nfxp = NFXPEstimator(data)
result_nfxp = nfxp.estimate()

# Display results using tabulate
true_params = model.params.theta_vector()
est_params = result_nfxp['theta']
param_names = ['alpha_1 (RC)', 'alpha_2 (theta_c)', 'alpha_3 (theta_s)', 'beta (discount)']

results_table = []
for i, name in enumerate(param_names):
    results_table.append([name, f"{true_params[i]:.4f}", f"{est_params[i]:.4f}"])

print("\nParameter Estimates:")
print(tabulate(results_table, headers=["Parameter", "True", "Estimated"], tablefmt="simple"))

stats_table = [
    ["Log-likelihood", f"{result_nfxp['log_likelihood']:.2f}"],
    ["Converged", result_nfxp['converged']],
    ["Iterations", result_nfxp['n_iterations']],
    ["Computation time (s)", f"{result_nfxp['computation_time']:.2f}"],
]
print("\nEstimation Statistics:")
print(tabulate(stats_table, headers=["Metric", "Value"], tablefmt="simple"))

## 3.2 Two-Step CCP Estimator (Hotz-Miller 1993)

**Key insight**: With finite dependence, we can avoid solving the DP.

### Two-Step Procedure

**Step 1: Estimate CCPs via Reduced-Form Logit**

Estimate a flexible logit model for replacement probabilities:
$$\hat{p}_0(x, z, s) = P(d=0|x, z, s) = \Lambda(\mathbf{x}'\hat{\gamma})$$

where $d=0$ is the **replacement** (base) action with utility normalized to zero.

**Step 2: Hotz-Miller Inversion**

The ex-ante value function can be recovered from CCPs using Hotz-Miller inversion:
$$\bar{V}(x) = \gamma - \log(p_0(x))$$

where:
- $\gamma \approx 0.5772$ is the Euler-Mascheroni constant
- $p_0(x) = P(d=0|x)$ is the probability of the **base action** (replacement)

**Why $p_0$ (not $p_1$)?** The Hotz-Miller inversion formula uses the probability of the action whose flow utility is normalized to zero. In the bus engine model, replacement ($d=0$) has $u_0 = 0$, so we use $p_0 = P(\text{replace})$.

### Future Value Difference

Define the future value difference:
$$FV(x) = \beta \cdot \big[E[\bar{V}(x') | x, d=1] - E[\bar{V}(x') | x, d=0]\big]$$

where:
- $d=1$ (continue): next-period mileage evolves from current $x$
- $d=0$ (replace): next-period mileage starts from $x=0$

### Second Stage GMM/MLE

The choice probability for continuing ($d=1$) is:
$$P(d=1|x) = \Lambda\big(u_1(x) + FV(x)\big) = \Lambda\big(\alpha_1 + \alpha_2 x + \alpha_3 s + FV(x)\big)$$

Estimate $\theta = (\alpha_1, \alpha_2, \alpha_3)$ via MLE with $FV(x)$ treated as a known regressor with coefficient constrained to 1 (or estimated freely as a specification check).


In [ ]:
class TwoStepCCPEstimator:
    """
    Two-step CCP estimator (Hotz-Miller 1993).

    Matches MATLAB replication convention (hetero10ns):
    - First stage: logit for P(replace) with base + type interactions + time dummies
    - FV = -log(P(replace)), WITHOUT β
    - Second stage: P(keep) = Λ(α₁ + α₂x + α₃s + β·FV + time_dummies)
    - β estimated as coefficient on FV
    - Last period excluded (terminal FV = 0)

    MATLAB CONVENTIONS (shellbusdatacomp420cond.m):
    - d=0: Replace (base, utility=0); d=1: Keep (flow utility = α₁ + α₂x + α₃s)
    - First stage models P(replace) via wlogitd with y2==0
    - FV = -log(P(replace)), no β, no γ (Euler cancels in difference)
    - Second stage: wlogit with Y=1 is keep, coefficient on FV estimates β
    """

    def __init__(self, data: pd.DataFrame, model: BusEngineModel):
        self.data = data
        self.model = model
        self.params = model.params
        self._P_continue = model._P_continue
        self._P_replace = model._P_replace

    def first_stage(self) -> np.ndarray:
        """
        Estimate CCPs via reduced-form logit.

        Models P(replace|x,z,s,t) with:
        - Base: [1, x/10, z, xz, x², z²]
        - Type interactions: [s, sx, sz, sxz, sx², sz²]
        - Time dummies for periods 1,...,T-1 (period 0 = baseline)

        MATLAB: wlogitd(b1, y2==0, xx, PType)
        """
        df = self.data
        p = self.params
        T = df['period'].max() + 1

        x_norm = df['x'].values / 10
        z = df['z'].values
        s = df['type_s'].values

        X_base = np.column_stack([
            np.ones(len(df)),
            x_norm, z, x_norm * z, x_norm**2, z**2,
            s, s * x_norm, s * z, s * x_norm * z, s * x_norm**2, s * z**2,
        ])

        # Time dummies (period 0 is baseline)
        td = np.zeros((len(df), T - 1))
        for t in range(1, T):
            td[:, t - 1] = (df['period'].values == t).astype(float)

        X = np.column_stack([X_base, td])

        # Model P(replace): y=1 when replace (MATLAB: wlogitd with y2==0)
        y = df['replace'].values.astype(float)

        def neg_ll(beta):
            prob = logit_prob(X @ beta)
            prob = np.clip(prob, 1e-10, 1 - 1e-10)
            return -np.sum(y * np.log(prob) + (1 - y) * np.log(1 - prob))

        beta0 = np.zeros(X.shape[1])
        result = minimize(neg_ll, beta0, method='BFGS', options={'disp': False})
        self.first_stage_beta = result.x
        self._T = T

        return self.first_stage_beta

    def compute_fv_terms(self, beta: np.ndarray) -> np.ndarray:
        """
        Compute FV terms WITHOUT β (MATLAB fvdata.m).

        MATLAB convention:
            FV1(:,s,t) = -log(P(replace|state, s, t))
            FV_obs(n,t) = (P_keep_trans - P_repl_trans) @ FV1(:, s, t+1)

        No β, no Euler constant (γ cancels in transition difference).
        β is estimated as a free coefficient in the second stage.
        """
        p = self.params
        n_x, n_z = p.n_x, p.n_z
        T = self._T
        tbin = n_x * n_z

        # Grid values: states ordered z-major (MATLAB: adj = x + (z-1)*xbin)
        xvalr = np.tile(p.x_grid / 10, n_z)
        zvalr = np.repeat(p.z_grid, n_x)

        # FV1[state, type, period] = -log(P(replace))
        FV1 = np.zeros((tbin, 2, T + 1))

        for t_pred in range(1, T):  # periods 1,...,T-1; FV at 0 and T stay 0
            td_grid = np.zeros((tbin, T - 1))
            td_grid[:, t_pred - 1] = 1.0

            for s in [0, 1]:
                X_grid = np.column_stack([
                    np.ones(tbin),
                    xvalr, zvalr, xvalr * zvalr, xvalr**2, zvalr**2,
                    np.full(tbin, s), s * xvalr, s * zvalr,
                    s * xvalr * zvalr, s * xvalr**2, s * zvalr**2,
                    td_grid
                ])

                p_replace = logit_prob(X_grid @ beta)
                p_replace = np.clip(p_replace, 1e-10, 1 - 1e-10)
                FV1[:, s, t_pred] = -np.log(p_replace)

        # Observation-level FV: (P_keep - P_replace) @ FV1[z_slice, s, t+1]
        df = self.data
        fv = np.zeros(len(df))

        for idx in df.index:
            z_idx = int(df.loc[idx, 'z_idx'])
            x_idx = int(df.loc[idx, 'x_idx'])
            t = int(df.loc[idx, 'period'])
            s = int(df.loc[idx, 'type_s'])

            z_start = z_idx * n_x
            z_end = (z_idx + 1) * n_x

            fv[idx] = (self._P_continue[z_idx][x_idx, :] - self._P_replace[z_idx]) \
                       @ FV1[z_start:z_end, s, t + 1]

        return fv

    def second_stage(self) -> dict:
        """
        Second stage: structural logit with β as free parameter.

        v₁ - v₀ = α₁ + α₂·x + α₃·s + β·FV + γ·td

        MATLAB: wlogit(bccp, y2, [xccp fvt1 td], PType)
        - xccp = [1, x*10, s]  (raw x, not x/10)
        - bccp = [α₁, α₂, α₃, β, time_dummies]
        - Excludes last period (FV=0 at terminal)
        - Time dummies for periods 1,...,T-2 (period 0 baseline, last excluded)
        """
        beta = self.first_stage()
        fv = self.compute_fv_terms(beta)

        df = self.data.copy()
        p = self.params
        T = self._T

        # Exclude last period (FV=0 at terminal)
        mask = df['period'].values < (T - 1)
        df_est = df[mask]
        fv_est = fv[mask]

        # Time dummies (period 0 baseline, periods 1...T-2)
        n_td = T - 2
        td = np.zeros((mask.sum(), n_td))
        periods_est = df_est['period'].values
        for t in range(1, T - 1):
            td[:, t - 1] = (periods_est == t).astype(float)

        # [1, x, s, FV, time_dummies] — x is raw mileage (MATLAB: x2*10)
        X2 = np.column_stack([
            np.ones(mask.sum()),
            df_est['x'].values,
            df_est['type_s'].values,
            fv_est,
            td
        ])

        # y=1 when keep (MATLAB: wlogit with Y=y2, Y=1 is keep)
        y2 = (df_est['replace'].values == 0).astype(float)

        def neg_ll(alpha):
            v = X2 @ alpha
            prob = logit_prob(v)
            prob = np.clip(prob, 1e-10, 1 - 1e-10)
            return -np.sum(y2 * np.log(prob) + (1 - y2) * np.log(1 - prob))

        # Initial: [α₁, α₂, α₃, β, time_dummies]
        alpha0 = np.concatenate([
            np.array([2.0, -0.1, 0.5, p.discount]),
            np.zeros(n_td)
        ])
        result = minimize(neg_ll, alpha0, method='BFGS')

        theta_hat = result.x[:4]  # [α₁, α₂, α₃, β]

        return {
            'theta': theta_hat,
            'time_dummy_coefs': result.x[4:],
            'log_likelihood': -result.fun,
            'first_stage_beta': beta
        }


In [ ]:
# Run Two-Step CCP estimation
print("="*70)
print("Two-Step CCP Estimation (types observed)")
print("="*70)

start = time.time()
twostep = TwoStepCCPEstimator(data, model)
result_twostep = twostep.second_stage()
elapsed = time.time() - start

# Display results using tabulate
true_params = model.params.theta_vector()
est_params = result_twostep['theta']
param_names = ['alpha_1 (intercept)', 'alpha_2 (mileage)', 'alpha_3 (type)', 'beta (discount)']

results_table = []
for i, name in enumerate(param_names):
    results_table.append([name, f"{true_params[i]:.4f}", f"{est_params[i]:.4f}"])

print("\nParameter Estimates:")
print(tabulate(results_table, headers=["Parameter", "True", "Estimated"], tablefmt="simple"))

stats_table = [
    ["Log-likelihood", f"{result_twostep['log_likelihood']:.2f}"],
    ["Computation time (s)", f"{elapsed:.2f}"],
]
print("\nEstimation Statistics:")
print(tabulate(stats_table, headers=["Metric", "Value"], tablefmt="simple"))


## 3.3 Comparison: NFXP vs Two-Stage CCP

### Key Differences

| Aspect | NFXP (Rust 1987) | Two-Stage CCP (Hotz-Miller 1993) |
|--------|------------------|----------------------------------|
| **Approach** | Nested optimization | Sequential estimation |
| **Inner loop** | Solve DP for each θ guess | None (CCPs estimated once from data) |
| **Outer loop** | MLE over θ | Regression of Hotz-Miller equation |
| **Computation** | $O(K \times T_{inner})$ where K=outer iterations | $O(N)$ where N=sample size |
| **Efficiency** | MLE efficient (achieves Cramér-Rao bound) | Less efficient (first-stage error propagates) |

### Algorithm Comparison

**NFXP (Nested Fixed Point)**:
```
For each θ candidate in optimization:
    1. Solve value function: V(x;θ) via contraction iteration
       - Initialize V⁰
       - Iterate: V^{k+1} = Γ(V^k; θ) until convergence
    2. Compute CCPs from V: p(x;θ) = Λ(v₁(x;θ) - v₀(x;θ))
    3. Evaluate likelihood: L(θ) = Π P(d_i|x_i; θ)
```

**Two-Stage CCP**:
```
Stage 1: Estimate CCPs non-parametrically
    - Flexible logit: p̂(x) = Λ(X'γ) with polynomials/interactions
    - No structural model needed

Stage 2: Use Hotz-Miller inversion
    - Compute ex-ante values: V̄(x) = γ - log(p̂(x))
    - Identify θ from: log(p̂/(1-p̂)) = u(x;θ) + β·FV(x)
    - Solve linear regression (fast!)
```

### Why Two-Stage Works (Finite Dependence Property)

The bus engine model has **finite dependence** because replacement ($d=0$) is a *renewal action*:
- After replacement, mileage resets to $x=0$
- The continuation value $E[V(x')|d=0]$ is the same for all starting states
- This enables the Hotz-Miller inversion to identify θ without solving the full DP

$$\bar{V}(x) - \bar{V}(0) = -\ln p_0(x) + \ln p_0(0)$$

### When to Use Each Method

| Scenario | Recommended Method |
|----------|-------------------|
| Small state space, need efficiency | NFXP |
| Large state space | Two-Stage CCP |
| Multiple specifications to test | Two-Stage CCP |
| Final estimates for publication | NFXP (for efficiency) |
| Robustness checks | Two-Stage CCP |

### Trade-off Summary

- **NFXP**: Slower but statistically efficient (MLE). Solves the full model, ensuring internal consistency.
- **Two-Stage**: Much faster but less efficient. First-stage estimation error propagates to second stage.

The efficiency loss from Two-Stage is typically small when:
1. Sample size is large (first-stage CCPs are precisely estimated)
2. State space is well-covered by data
3. The finite dependence property holds


In [ ]:
# Summary comparison
print("="*70)
print("Comparison: Estimation Without Heterogeneity")
print("="*70)

true_params = model.params.theta_vector()
nfxp_params = result_nfxp['theta']
twostep_params = result_twostep['theta']
param_names = ['alpha_1 (intercept)', 'alpha_2 (mileage)', 'alpha_3 (type)', 'beta (discount)']

comparison_table = []
for i, name in enumerate(param_names):
    comparison_table.append([
        name,
        f"{true_params[i]:.4f}",
        f"{nfxp_params[i]:.4f}",
        f"{twostep_params[i]:.4f}"
    ])

print("\nParameter Estimates:")
print(tabulate(comparison_table, headers=["Parameter", "True", "NFXP", "Two-Step"], tablefmt="simple"))

timing_table = [
    ["NFXP", f"{result_nfxp['computation_time']:.2f}", f"{result_nfxp['log_likelihood']:.2f}"],
    ["Two-Step", f"{elapsed:.2f}", f"{result_twostep['log_likelihood']:.2f}"],
]
print("\nPerformance Comparison:")
print(tabulate(timing_table, headers=["Method", "Time (s)", "Log-Likelihood"], tablefmt="simple"))


---
# Part 4: Estimation With Unobserved Heterogeneity

Now suppose type $s_i$ is **unobserved**. We only see:
- Mileage $x_{it}$
- Z parameter $z_i$ (bus-specific)
- Replacement decisions $d_{it}$

**Challenge**: We need to integrate over the unknown type distribution.

## The Likelihood

$$\mathcal{L}(\theta) = \sum_i \log \left[ \sum_{s=0}^1 \pi_s \prod_t P(d_{it} | x_{it}, s; \theta) \right]$$

where $\pi_s = P(s_i = s)$ is the type probability.

## 4.1 Why Two-Step CCP Fails

**The problem**: In the first stage, we estimate
$$\hat{p}(x, z) = \frac{1}{N_{xz}} \sum_{i: x_i = x, z_i = z} \mathbf{1}\{d_i = 1\}$$

But this pools across types:
$$\hat{p}(x, z) \approx \pi_0 \cdot p(x, z | s=0) + \pi_1 \cdot p(x, z | s=1)$$

**We need type-specific CCPs** $p(x, z | s)$, but we don't know $s$!

To get type-specific CCPs, we need posterior type probabilities:
$$P(s_i = s | \text{data}_i; \theta) \propto \pi_s \prod_t P(d_{it} | x_{it}, s; \theta)$$

But to compute this, we need $\theta$... **Chicken and egg problem!**

**Solution**: Iterate via EM algorithm.

In [ ]:
# Create data without type labels (simulating unobserved heterogeneity)
data_unobs = data.drop(columns=['type_s']).copy()
print("Data without type labels:")
print(data_unobs.head())

## 4.2 NFXP + EM

The EM algorithm iterates between:

**E-step**: Given current $\theta^k$, compute posterior type probabilities
$$\tau_{is}^k = P(s_i = s | Y_i, X_i; \theta^k) = \frac{\pi_s \prod_t P(d_{it} | x_{it}, s; \theta^k)}{\sum_{s'} \pi_{s'} \prod_t P(d_{it} | x_{it}, s'; \theta^k)}$$

**M-step**: Given $\tau^k$, maximize expected complete-data likelihood
$$\theta^{k+1} = \arg\max_\theta \sum_{i,s} \tau_{is}^k \sum_t \log P(d_{it} | x_{it}, s; \theta)$$

In NFXP+EM, the M-step requires solving the DP at each parameter guess (nested within optimization).

In [ ]:
class NFXPEMEstimator:
    """
    NFXP with EM for unobserved heterogeneity.

    Inner loop: For each theta guess, solve DP via Bellman value iteration
    (stationary model), compute likelihood using finite dependence.
    Outer loop: EM iterations over type probabilities.

    Based on MATLAB nohetero/likebusML4.m structure, adapted to stationary
    model to match the Python data generating process (BusEngineModel.solve()
    uses CCP fixed-point iteration, producing stationary choice probabilities).

    MATLAB note: likebusML4.m uses finite-horizon backward induction because
    genbus4.m generates data from a finite-horizon model. Our Python Simulator
    generates from the stationary (infinite-horizon) model, so we use Bellman
    iteration to convergence instead. The likelihood structure is otherwise
    identical: finite dependence with FV = beta * log(exp(u_keep + E[FV']) +
    exp(E[FV'|replace])).

    Key design choices:
      - Bellman value iteration to convergence (stationary FV)
      - Finite dependence in likelihood (matching likebusML4.m)
      - beta via logit transform: beta = sigma(alpha_4) for unconstrained opt
      - Initial conditions correction (intcond / intcondP)
    """

    def __init__(self, data: pd.DataFrame, base_params: BusEngineParams = None):
        self.data = data.copy()
        self.base_params = base_params or BusEngineParams()
        p = self.base_params

        self.bus_ids = sorted(data['bus_id'].unique())
        self.n_buses = len(self.bus_ids)
        N = self.n_buses
        self.T = data['period'].max() + 1

        # Per-bus data
        self.bus_data = {}
        for bus_id in self.bus_ids:
            mask = data['bus_id'] == bus_id
            self.bus_data[bus_id] = {
                'x': data.loc[mask, 'x'].values,
                'x_idx': data.loc[mask, 'x_idx'].values,
                'z_idx': data.loc[mask, 'z_idx'].values[0],
                'replace': data.loc[mask, 'replace'].values,
            }

        # Transition matrices
        self._P_continue = []
        self._P_replace = []
        for z in p.z_grid:
            P = build_transition_matrix(z, p.x_grid)
            self._P_continue.append(P)
            self._P_replace.append(P[0, :])

        # Initial conditions: [1, x_0, z]
        initial_data = data[data['period'] == 0].set_index('bus_id').loc[self.bus_ids]
        self.intcondX = np.column_stack([
            np.ones(N),
            initial_data['x'].values,
            initial_data['z'].values,
        ])

    # =========================================================================
    # Bellman Value Iteration (stationary model)
    # =========================================================================

    def _solve_dp(self, alpha_raw: np.ndarray) -> Tuple[np.ndarray, float]:
        """
        Solve for stationary value function via Bellman iteration.

        Iterates: FV(x,z,s) = beta * log(exp(u_keep + E[FV'|keep]) + exp(E[FV'|replace]))
        until convergence.

        Matches likebusML4.m backward induction structure but iterated to
        convergence for consistency with stationary DGP.

        alpha_raw = [alpha_1, alpha_2, alpha_3, alpha_4_logit] where beta = sigma(alpha_4_logit).

        Returns (FV, Beta) where FV has shape (n_z, n_x, 2) — stationary (no time dimension).
        """
        p = self.base_params
        n_x, n_z = p.n_x, p.n_z

        Beta = logit_prob(alpha_raw[3])

        FV = np.zeros((n_z, n_x, 2))
        u_base = alpha_raw[0] + alpha_raw[1] * p.x_grid  # (n_x,)

        for iteration in range(500):
            FV_new = np.zeros_like(FV)

            for s in range(2):
                u1_flow = u_base + alpha_raw[2] * s
                for z_idx in range(n_z):
                    P_cont = self._P_continue[z_idx]
                    P_repl = self._P_replace[z_idx]
                    FV_curr = FV[z_idx, :, s]

                    util_keep = u1_flow + P_cont @ FV_curr
                    util_replace = P_repl @ FV_curr  # scalar, broadcast

                    max_u = np.maximum(util_keep, util_replace)
                    FV_new[z_idx, :, s] = Beta * (
                        max_u + np.log(np.exp(util_keep - max_u) + np.exp(util_replace - max_u))
                    )

            diff = np.max(np.abs(FV_new - FV))
            FV = FV_new
            if diff < 1e-10:
                break

        return FV, Beta

    # =========================================================================
    # Type-specific likelihoods
    # =========================================================================

    def _compute_type_likelihoods(self, alpha_raw: np.ndarray) -> np.ndarray:
        """
        Bus-level likelihood under each type. Returns base: (N, 2).

        Uses stationary FV (no time subscript). Finite dependence structure
        matches likebusML4.m: util1 = alpha_1 + alpha_2*x + alpha_3*s +
        (P_cont - P_repl) @ FV(z, s).
        """
        N, T = self.n_buses, self.T
        FV, Beta = self._solve_dp(alpha_raw)

        base = np.zeros((N, 2))

        for i, bus_id in enumerate(self.bus_ids):
            bd = self.bus_data[bus_id]
            z_idx = bd['z_idx']

            for s in range(2):
                log_lik = 0.0
                # Precompute FV-related terms for this (z, s)
                FV_zs = FV[z_idx, :, s]  # stationary — same for all t
                P_repl_row = self._P_replace[z_idx]
                ev_replace = P_repl_row @ FV_zs

                for t in range(T):
                    x_idx = bd['x_idx'][t]
                    P_cont_row = self._P_continue[z_idx][x_idx, :]
                    ev_cont = P_cont_row @ FV_zs
                    fd = ev_cont - ev_replace

                    util1 = alpha_raw[0] + alpha_raw[1] * bd['x'][t] + alpha_raw[2] * s + fd
                    y_keep = 1.0 - bd['replace'][t]

                    log1pexp = util1 if util1 > 20 else np.log1p(np.exp(min(util1, 500)))
                    log_lik -= (log1pexp - y_keep * util1)

                base[i, s] = np.exp(np.clip(log_lik, -500, 0))

        return base

    def _nfxp_weighted_nll(self, alpha_raw: np.ndarray, PType: np.ndarray) -> float:
        """Weighted NFXP NLL for M-step. Uses stationary FV."""
        N, T = self.n_buses, self.T
        FV, Beta = self._solve_dp(alpha_raw)

        nll = 0.0
        for i, bus_id in enumerate(self.bus_ids):
            bd = self.bus_data[bus_id]
            z_idx = bd['z_idx']

            for s in range(2):
                w = PType[i, s]
                if w < 1e-10:
                    continue

                FV_zs = FV[z_idx, :, s]
                P_repl_row = self._P_replace[z_idx]
                ev_replace = P_repl_row @ FV_zs

                for t in range(T):
                    x_idx = bd['x_idx'][t]
                    P_cont_row = self._P_continue[z_idx][x_idx, :]
                    ev_cont = P_cont_row @ FV_zs
                    fd = ev_cont - ev_replace

                    util1 = alpha_raw[0] + alpha_raw[1] * bd['x'][t] + alpha_raw[2] * s + fd
                    y_keep = 1.0 - bd['replace'][t]
                    log1pexp = util1 if util1 > 20 else np.log1p(np.exp(min(util1, 500)))
                    nll += w * (log1pexp - y_keep * util1)

        return nll

    # =========================================================================
    # Initial conditions
    # =========================================================================

    def _intcond(self, gamma: np.ndarray, base: np.ndarray) -> float:
        # Note: MATLAB intcond.m divides by 200 (hardcoded); we divide by N.
        # Same optimum, different scale — does not affect parameter estimates.
        p0 = logit_prob(self.intcondX @ gamma)
        p = np.column_stack([p0, 1 - p0])
        marginal = np.sum(p * base, axis=1)
        return -np.sum(np.log(np.maximum(marginal, 1e-300))) / self.n_buses

    def _intcondP(self, gamma: np.ndarray, base: np.ndarray) -> np.ndarray:
        p0 = logit_prob(self.intcondX @ gamma)
        p = np.column_stack([p0, 1 - p0])
        joint = base * p
        marginal = np.sum(joint, axis=1, keepdims=True)
        return joint / np.maximum(marginal, 1e-300)

    # =========================================================================
    # EM Algorithm
    # =========================================================================

    def estimate(self, theta0: np.ndarray = None, max_iter: int = 50,
                 tol: float = 1e-7, fix_discount: bool = True) -> dict:
        """
        Run NFXP + EM.

        Parameters use logit transform for beta:
          alpha_raw = [alpha_1, alpha_2, alpha_3, logit(beta)]
        Results report actual beta.

        fix_discount : bool
            If True (default), fix beta at base_params.discount and optimize only (alpha_1, alpha_2, alpha_3).
            If False, estimate beta jointly.
        """
        p = self.base_params

        if theta0 is None:
            theta0 = np.array([p.theta0_intercept, p.theta0_mileage, p.theta0_type, p.discount])

        beta0 = np.clip(theta0[3], 0.01, 0.99)
        alpha_raw = np.array([theta0[0], theta0[1], theta0[2],
                              np.log(beta0) - np.log(1 - beta0)])

        gamma = np.zeros(3)
        gamma_start = np.zeros(3)
        lp_history = []
        history = []
        start_time = time.time()

        for j in range(max_iter):
            # -- E-step --
            base = self._compute_type_likelihoods(alpha_raw)

            # Initial conditions
            if j > 1:
                ic_start = gamma_start if j < 50 else gamma
                res_ic = minimize(self._intcond, ic_start, args=(base,),
                                  method='BFGS', options={'disp': False})
                gamma = res_ic.x

            ll = self._intcond(gamma, base)
            lp_history.append(ll)

            PType = self._intcondP(gamma, base)
            tau = PType[:, 1]
            pi_new = 1 - tau.mean()

            # -- M-step --
            if fix_discount:
                beta_logit_fixed = alpha_raw[3]
                def m_obj_3(a3):
                    a_full = np.array([a3[0], a3[1], a3[2], beta_logit_fixed])
                    return self._nfxp_weighted_nll(a_full, PType)

                res_m = minimize(m_obj_3, alpha_raw[:3], method='L-BFGS-B',
                                 options={'maxiter': 200, 'disp': False})
                alpha_raw_new = np.array([res_m.x[0], res_m.x[1], res_m.x[2], beta_logit_fixed])
            else:
                def m_obj(a):
                    return self._nfxp_weighted_nll(a, PType)

                res_m = minimize(m_obj, alpha_raw, method='L-BFGS-B',
                                 options={'maxiter': 200, 'disp': False})
                alpha_raw_new = res_m.x

            # Convert back for reporting
            beta_est = logit_prob(alpha_raw_new[3])
            theta_report = np.array([alpha_raw_new[0], alpha_raw_new[1],
                                     alpha_raw_new[2], beta_est])

            history.append({
                'theta': theta_report.copy(), 'pi': pi_new,
                'll': ll, 'gamma': gamma.copy()
            })

            print(f"Iter {j+1}: theta={theta_report.round(3)}, "
                  f"pi={pi_new:.3f}, gamma={gamma.round(3)}, LL={ll:.4f}")

            # Convergence
            diff = np.max(np.abs(alpha_raw_new - alpha_raw))
            if diff < tol and j > 5:
                print(f"Converged in {j+1} iterations")
                break

            if len(lp_history) > 26:
                c1 = abs((lp_history[-1] - lp_history[-26]) / max(abs(lp_history[-1]), 1e-300))
                c2 = abs((lp_history[-2] - lp_history[-27]) / max(abs(lp_history[-2]), 1e-300))
                if c1 < tol and c2 < tol:
                    print(f"Converged (LL criterion) in {j+1} iterations")
                    break

            alpha_raw = alpha_raw_new

        return {
            'theta': theta_report,
            'pi': pi_new,
            'gamma': gamma,
            'tau': tau,
            'log_likelihood': ll,
            'n_iterations': j + 1,
            'computation_time': time.time() - start_time,
            'history': history
        }


In [ ]:
# Run NFXP+EM (full sample, matching replication N=2000)
print("="*70)
print("NFXP + EM Estimation (types unobserved)")
print("="*70)

data_unobs = data.drop(columns=['type_s'])

config_table = [
    ["Buses used", data_unobs['bus_id'].nunique()],
    ["Total observations", len(data_unobs)],
    ["Periods", data_unobs['period'].nunique()],
]
print("\nConfiguration:")
print(tabulate(config_table, headers=["Parameter", "Value"], tablefmt="simple"))

print("\nNote: NFXP+EM is slow — each iteration solves the full DP via backward induction.")
print("Expected: ~60x slower than CCP-EM.\n")

nfxp_em = NFXPEMEstimator(data_unobs, model.params)
result_nfxp_em = nfxp_em.estimate(max_iter=30)

# Display results
true_params = model.params.theta_vector()
est_params = result_nfxp_em['theta']
param_names = ['alpha_1 (intercept)', 'alpha_2 (mileage)', 'alpha_3 (type)', 'beta (discount)']

results_table = []
for i, name in enumerate(param_names):
    results_table.append([name, f"{true_params[i]:.4f}", f"{est_params[i]:.4f}"])
results_table.append(['pi (type prob)', f"{model.params.type_prob:.4f}", f"{result_nfxp_em['pi']:.4f}"])

print("\nParameter Estimates:")
print(tabulate(results_table, headers=["Parameter", "True", "Estimated"], tablefmt="simple"))

# Estimation statistics
stats_table = [
    ["Iterations", result_nfxp_em['n_iterations']],
    ["Computation time (s)", f"{result_nfxp_em['computation_time']:.2f}"],
]
print("\nEstimation Statistics:")
print(tabulate(stats_table, headers=["Metric", "Value"], tablefmt="simple"))


## 4.3 CCP-EM with Data Updating

**Key innovation of Arcidiacono & Miller (2011)**: Replace the expensive M-step (solving DP) with CCP-based estimation.

### CCP-Data Approach

In the M-step, instead of solving the DP:

1. **Update CCPs directly from data** using posterior weights:
   $$\hat{p}^{k+1}(x, s) = \frac{\sum_{i,t} \tau_{is}^k \mathbf{1}\{x_{it}=x, d_{it}=1\}}{\sum_{i,t} \tau_{is}^k \mathbf{1}\{x_{it}=x\}}$$

2. **Estimate structural parameters** via weighted regression with Hotz-Miller inversion.

This avoids solving the DP entirely!

In [ ]:
class CCPEMDataEstimator:
    """
    CCP-EM with Data updating + Initial Conditions.

    Exact Python translation of MATLAB hetero10ns/shellbusdatacomp420cond.m.

    EM loop structure:
      E-step: likeCCP → intcond → intcondP → PType
      M-step 1: wlogitd (re-estimate reduced-form with PType weights)
      M-step 2: fvdata (recompute FV from updated reduced-form)
      M-step 3: wlogit (re-estimate structural with PType weights)

    Key design choices matching MATLAB:
      - Single pooled reduced-form logit with type interactions (not separate type logits)
      - FV from data-based CCPs evaluated on grid (β NOT in FV)
      - likeCCP (structural model) for type classification (not reduced-form)
      - Time-varying FV via period-specific dummies in reduced-form
      - Last period excluded from structural estimation
      - Initial conditions: P(type=0|x₀,z) = Λ(γ'w)
    """

    def __init__(self, data: pd.DataFrame, base_params: BusEngineParams = None):
        self.data = data.copy()
        self.base_params = base_params or BusEngineParams()
        p = self.base_params

        # ── Bus-level indexing ──
        self.bus_ids = sorted(data['bus_id'].unique())
        self.n_buses = len(self.bus_ids)
        N = self.n_buses
        self.T = data['period'].max() + 1
        T = self.T

        # Per-bus data
        self.bus_data = {}
        for bus_id in self.bus_ids:
            mask = data['bus_id'] == bus_id
            self.bus_data[bus_id] = {
                'x': data.loc[mask, 'x'].values,
                'x_idx': data.loc[mask, 'x_idx'].values,
                'z': data.loc[mask, 'z'].values[0],
                'z_idx': data.loc[mask, 'z_idx'].values[0],
                'replace': data.loc[mask, 'replace'].values,
                'periods': data.loc[mask, 'period'].values,
            }

        # ── Transition matrices ──
        self._P_continue = []  # P(x'|x,z, keep)
        self._P_replace = []   # P(x'|0,z, replace) = first row
        for z in p.z_grid:
            P = build_transition_matrix(z, p.x_grid)
            self._P_continue.append(P)
            self._P_replace.append(P[0, :])

        # ── Build stacked data arrays ──
        # MATLAB ordering: y2 = reshape(Y, N*T, 1) then [y2; y2]
        # Column-major reshape = time-major: (bus1_t1, bus2_t1, ..., busN_t1, bus1_t2, ...)
        # For simplicity we use bus-major: (bus1_t0..tT, bus2_t0..tT, ...) — same result
        # since all weighted sums are permutation-invariant.

        y_list, x_list, z_list, xidx_list, zidx_list, t_list = [], [], [], [], [], []
        for bus_id in self.bus_ids:
            bd = self.bus_data[bus_id]
            for t in range(T):
                y_list.append(bd['replace'][t])
                x_list.append(bd['x'][t])
                xidx_list.append(bd['x_idx'][t])
                z_list.append(bd['z'])
                zidx_list.append(bd['z_idx'])
                t_list.append(t)

        y_s = np.array(y_list)            # N*T
        x_s = np.array(x_list)
        z_s = np.array(z_list)
        xidx_s = np.array(xidx_list)
        zidx_s = np.array(zidx_list)
        t_s = np.array(t_list)

        # Stack: [type0; type1]
        self.y2 = np.concatenate([y_s, y_s])           # 2*N*T
        self.y_replace = (self.y2 == 1).astype(float)   # replace indicator (MATLAB: y2==0 where y2=1=keep)
        # MATLAB convention: Y=1=keep, y2==0=replace. Our data: replace=1=replace.
        # So our replace indicator = self.y2 directly. MATLAB's (y2==0) = our replace.
        # For wlogitd modeling P(replace|X)=σ(Xb): Y_wlogit = replace indicator.

        x2 = np.concatenate([x_s, x_s]) / 10.0   # MATLAB: x2 = X/10
        z2 = np.concatenate([z_s, z_s])
        self.s2 = np.concatenate([np.zeros(N*T), np.ones(N*T)])
        self.t2 = np.concatenate([t_s, t_s])
        self.xidx2 = np.concatenate([xidx_s, xidx_s])
        self.zidx2 = np.concatenate([zidx_s, zidx_s])

        # ── Time dummies (T-1 columns) ──
        # MATLAB: td(:,t) = (t2*10-1 == t) for t=1..T-1
        # In 0-indexed: td[:,k] = (period == k+1) for k=0..T-2
        # Period 0 is reference.
        self.td = np.zeros((2*N*T, T-1))
        for k in range(T-1):
            self.td[:, k] = (self.t2 == k+1).astype(float)

        # ── Reduced-form design matrix ──
        # MATLAB: xx = [1, x2, z2, x2.*z2, x2.^2, z2.^2, s2, s2.*x2, s2.*z2,
        #               s2.*x2.*z2, s2.*x2.^2, s2.*z2.^2, td]
        self.xx = np.column_stack([
            np.ones(2*N*T),
            x2, z2, x2*z2, x2**2, z2**2,
            self.s2, self.s2*x2, self.s2*z2,
            self.s2*x2*z2, self.s2*x2**2, self.s2*z2**2,
            self.td
        ])

        # ── Grid design matrix for FV (RX1) ──
        # MATLAB: xvalr = kron(ones(zbin,1), xval)/10, zvalr = kron(zval, ones(xbin,1))
        # RX1 = [1, xvalr, zvalr, xvalr.*zvalr, xvalr.^2, zvalr.^2]
        n_x, n_z = p.n_x, p.n_z
        xvalr = np.tile(p.x_grid / 10.0, n_z)      # repeat x_grid for each z
        zvalr = np.repeat(p.z_grid, n_x)             # repeat each z for all x
        self.RX1 = np.column_stack([
            np.ones(n_x * n_z),
            xvalr, zvalr, xvalr*zvalr, xvalr**2, zvalr**2
        ])
        self.tbin = n_x * n_z

        # ── Structural regressors ──
        # MATLAB: xccp = [1, x2*10, s2] = [1, X_original, s]
        self.xccp = np.column_stack([np.ones(2*N*T), x2*10.0, self.s2])

        # ── Index: exclude last period ──
        # MATLAB: index = t2 < 1 (since t2 = period/10, this means period < T)
        self.index = self.t2 < (T - 1)

        # ── Initial conditions ──
        # MATLAB: intcondX = [1, X(1:N,1), Z(1:N,1)]
        initial_data = data[data['period'] == 0].set_index('bus_id').loc[self.bus_ids]
        self.intcondX = np.column_stack([
            np.ones(N),
            initial_data['x'].values,
            initial_data['z'].values,
        ])

    # =========================================================================
    # MATLAB function translations
    # =========================================================================

    @staticmethod
    def _plogit(b: np.ndarray, X: np.ndarray) -> np.ndarray:
        """plogit.m: P(replace|X) = exp(Xb)/(1+exp(Xb)) = σ(Xb)"""
        return logit_prob(X @ b)

    @staticmethod
    def _wlogitd(b: np.ndarray, Y: np.ndarray, X: np.ndarray, P: np.ndarray):
        """
        wlogitd.m: Weighted logit NLL with gradient.
        Models P(Y=1|X) = σ(Xb).
        NLL = P' @ (log(1+exp(Xb)) - Y*Xb)
        grad = (P*(σ(Xb)-Y))' @ X
        """
        U = X @ b
        # Numerically stable log(1+exp(U))
        log1pexp = np.where(U > 20, U, np.log1p(np.exp(np.clip(U, -500, 20))))
        nll = P @ (log1pexp - Y * U)
        sigma = logit_prob(U)
        grad = (P * (sigma - Y)) @ X
        return nll, grad

    @staticmethod
    def _wlogit(b: np.ndarray, Y: np.ndarray, X: np.ndarray, P: np.ndarray) -> float:
        """wlogit.m: Weighted logit NLL (no gradient). Like = P'*(log(1+exp(Xb))-Y.*Xb)"""
        U = X @ b
        log1pexp = np.where(U > 20, U, np.log1p(np.exp(np.clip(U, -500, 20))))
        return P @ (log1pexp - Y * U)

    @staticmethod
    def _likeCCP(b: np.ndarray, Y: np.ndarray, X: np.ndarray) -> np.ndarray:
        """
        likeCCP.m: Per-observation structural likelihood.
        Like = (Y*exp(Xb) + (1-Y)) / (1+exp(Xb))
        = Y*σ(Xb) + (1-Y)*(1-σ(Xb))
        where Y=1 means keep (the action modeled by σ(Xb)).
        """
        U = X @ b
        expU = np.exp(np.clip(U, -500, 500))
        return (Y * expU + (1 - Y)) / (1 + expU)

    def _intcond(self, gamma: np.ndarray, base: np.ndarray) -> float:
        """intcond.m: NLL for initial conditions. Like=-sum(log(sum(p.*like,2)))/N"""
        # Note: MATLAB intcond.m divides by 200 (hardcoded); we divide by N.
        # Same optimum, different scale — does not affect parameter estimates.
        p0 = logit_prob(self.intcondX @ gamma)
        p = np.column_stack([p0, 1 - p0])
        marginal = np.sum(p * base, axis=1)
        return -np.sum(np.log(np.maximum(marginal, 1e-300))) / self.n_buses

    def _intcondP(self, gamma: np.ndarray, base: np.ndarray) -> np.ndarray:
        """intcondP.m: Posterior type probs. PType=(like.*p)./(sum(like.*p,2))"""
        p0 = logit_prob(self.intcondX @ gamma)
        p = np.column_stack([p0, 1 - p0])
        joint = base * p
        marginal = np.sum(joint, axis=1, keepdims=True)
        return joint / np.maximum(marginal, 1e-300)

    def _fvdata(self, b1: np.ndarray) -> np.ndarray:
        """
        fvdata.m: Compute FV terms from reduced-form logit b1.

        1. For each period t (0-indexed: 1..T-1) and type s:
           Build grid regressors [RX1, s*RX1, td_grid]
           p0 = plogit(b1, X_grid) = P(replace|grid,s,t)
           FV1[grid,s,t] = -log(p0)

        2. For each (bus, period, type):
           FVT1[n,t,s] = (P_continue[x_n_t] - P_replace[x=0]) @ FV1[z_grid,s,t+1]

        Returns stacked fvt1 of length 2*N*T.
        """
        p = self.base_params
        n_x, n_z = p.n_x, p.n_z
        N, T = self.n_buses, self.T
        tbin = self.tbin

        # FV1: (tbin, 2_types, T+1_periods) — 0-indexed periods, period T is terminal (=0)
        FV1 = np.zeros((tbin, 2, T + 1))
        td_grid = np.zeros((tbin, T - 1))

        # MATLAB loop: for t=2:T (1-indexed) → 0-indexed: t=1..T-1
        for t_0idx in range(1, T):
            # Set one-hot time dummy for this period
            td_grid[:, :] = 0
            td_grid[:, t_0idx - 1] = 1.0

            for s in range(2):
                X_grid = np.column_stack([self.RX1, s * self.RX1, td_grid])
                p0 = self._plogit(b1, X_grid)
                p0 = np.clip(p0, 1e-10, 1 - 1e-10)
                FV1[:, s, t_0idx] = -np.log(p0)

        # Compute FVT1 for each (bus, period, type)
        FVT1 = np.zeros((N, T, 2))

        for i, bus_id in enumerate(self.bus_ids):
            bd = self.bus_data[bus_id]
            z_idx = bd['z_idx']
            z_start = z_idx * n_x       # grid slice start for this z
            z_end = z_start + n_x        # grid slice end

            # P_replace row = transition from x=0 for this z
            P_repl_row = self._P_continue[z_idx][0, :]  # = P(x'|x=0, z)

            for t in range(T):
                x_idx = bd['x_idx'][t]
                P_cont_row = self._P_continue[z_idx][x_idx, :]  # P(x'|x,z,keep)

                for s in range(2):
                    # Finite dependence: (P_cont - P_repl) @ FV1[z_slice, s, t+1]
                    FVT1[i, t, s] = (P_cont_row - P_repl_row) @ FV1[z_start:z_end, s, t + 1]

        # Stack: [all type0 (N*T); all type1 (N*T)]
        fvt1 = np.concatenate([FVT1[:, :, 0].flatten(), FVT1[:, :, 1].flatten()])
        return fvt1

    # =========================================================================
    # EM Algorithm
    # =========================================================================

    def estimate(self, theta0: np.ndarray = None, max_iter: int = 200,
                 tol: float = 1e-7) -> dict:
        """
        Run CCP-EM with data updating + initial conditions.

        Exact match of MATLAB shellbusdatacomp420cond.m lines 83-168.
        """
        p = self.base_params
        N, T = self.n_buses, self.T
        idx = self.index  # exclude last period

        # Starting values
        # MATLAB: alphac = [2.233; -.1339; .4; .9115]
        if theta0 is None:
            theta0 = np.array([p.theta0_intercept, p.theta0_mileage, p.theta0_type, p.discount])

        # MATLAB: bccp = [alphac(1:4); zeros(T2-2,1)]
        n_td_struct = max(T - 2, 0)
        bccp = np.concatenate([theta0[:4], np.zeros(n_td_struct)])

        # Initial PType = 0.5 for all observations
        PType_obs = 0.5 * np.ones(2 * N * T)

        # ── Pre-loop: initial reduced-form estimate + FV ──
        b1 = np.zeros(self.xx.shape[1])

        # MATLAB: [b1]=fminunc('wlogitd',b1,o2,y2==0,xx,PType)
        def wlogitd_obj(b):
            nll, grad = self._wlogitd(b, self.y_replace, self.xx, PType_obs)
            return nll, grad

        res = minimize(wlogitd_obj, b1, method='BFGS', jac=True,
                       options={'disp': False, 'maxiter': 200})
        b1 = res.x

        # MATLAB: [fvt1]=fvdata(b1,RX1,...)
        fvt1 = self._fvdata(b1)

        # ── EM loop ──
        gamma = np.zeros(3)
        gamma_start = np.zeros(3)
        lp_history = []
        history = []
        start_time = time.time()

        for j in range(max_iter):
            # ── E-step: type classification via structural likelihood ──

            # MATLAB: Like=likeCCP(bccp,y2(index==1),[xccp(index==1,:) fvt1(index==1) td(index==1,1:T2-2)])
            X_struct = np.column_stack([
                self.xccp[idx],
                fvt1[idx],
                self.td[idx, :n_td_struct]   # T-2 structural time dummies
            ])
            y_keep = (1 - self.y_replace[idx]).astype(float)  # Y=1=keep for likeCCP
            Like = self._likeCCP(bccp, y_keep, X_struct)

            # MATLAB: Like2=reshape(Like,N,T2-1,2); base=squeeze(prod(Like2,2))
            half = int(idx.sum()) // 2
            like0 = Like[:half].reshape(N, T - 1)
            like1 = Like[half:].reshape(N, T - 1)
            base0 = np.prod(np.clip(like0, 1e-300, None), axis=1)
            base1 = np.prod(np.clip(like1, 1e-300, None), axis=1)
            base = np.column_stack([base0, base1])

            # ── Initial conditions ──
            # MATLAB: if j>1, estimate gamma
            if j > 1:
                ic_start = gamma_start if j < 50 else gamma
                res_ic = minimize(self._intcond, ic_start, args=(base,),
                                  method='BFGS', options={'disp': False})
                gamma = res_ic.x

            ll = self._intcond(gamma, base)
            lp_history.append(ll)

            # ── Posterior type probabilities ──
            PType = self._intcondP(gamma, base)
            tau = PType[:, 1]
            pi_new = 1 - tau.mean()

            # Expand PType to per-observation weights
            # MATLAB: PType=kron(ones(T2,1),PType); PType=reshape(PType,N*T2*2,1)
            PType_obs = np.zeros(2 * N * T)
            for i in range(N):
                for t in range(T):
                    PType_obs[i * T + t] = PType[i, 0]           # type 0 block
                    PType_obs[N * T + i * T + t] = PType[i, 1]   # type 1 block

            # ── M-step 1: re-estimate reduced-form logit ──
            # MATLAB: [b1]=fminunc('wlogitd',b1,o2,y2==0,xx,PType)
            def wlogitd_obj_em(b):
                nll, grad = self._wlogitd(b, self.y_replace, self.xx, PType_obs)
                return nll, grad

            res_rf = minimize(wlogitd_obj_em, b1, method='BFGS', jac=True,
                              options={'disp': False, 'maxiter': 200})
            b1 = res_rf.x

            # ── M-step 2: recompute FV ──
            fvt1 = self._fvdata(b1)

            # ── M-step 3: re-estimate structural parameters ──
            # MATLAB: [bccp]=fminunc('wlogit',bccp,o1,y2(idx),[xccp(idx) fvt1(idx) td(idx,1:T-2)],PType(idx))
            X_struct_full = np.column_stack([
                self.xccp[idx],
                fvt1[idx],
                self.td[idx, :n_td_struct]
            ])
            PType_idx = PType_obs[idx]

            def wlogit_obj(b):
                return self._wlogit(b, y_keep, X_struct_full, PType_idx)

            res_struct = minimize(wlogit_obj, bccp, method='BFGS',
                                  options={'disp': False, 'maxiter': 200})
            bccp = res_struct.x
            theta_new = bccp[:4]

            history.append({
                'theta': theta_new.copy(), 'pi': pi_new,
                'll': ll, 'gamma': gamma.copy()
            })

            if j % 5 == 0:
                print(f"Iter {j+1}: θ={theta_new.round(3)}, "
                      f"π={pi_new:.3f}, γ={gamma.round(3)}, LL={ll:.4f}")

            # ── Convergence (MATLAB: 26-lag relative change) ──
            if len(lp_history) > 26:
                c1 = abs((lp_history[-1] - lp_history[-26]) / max(abs(lp_history[-1]), 1e-300))
                c2 = abs((lp_history[-2] - lp_history[-27]) / max(abs(lp_history[-2]), 1e-300))
                if c1 < tol and c2 < tol:
                    print(f"Converged in {j+1} iterations")
                    break

        return {
            'theta': theta_new,
            'pi': pi_new,
            'gamma': gamma,
            'tau': tau,
            'bccp': bccp,
            'b1': b1,
            'log_likelihood': ll,
            'n_iterations': j + 1,
            'computation_time': time.time() - start_time,
            'history': history
        }


In [ ]:
# Run CCP-EM with Data updating
print("="*70)
print("CCP-EM (Data Updating) — Arcidiacono & Miller (2011)")
print("="*70)

data_unobs = data.drop(columns=['type_s'])

ccpem_data = CCPEMDataEstimator(data_unobs, model.params)
result_ccpem_data = ccpem_data.estimate()

# Display results
true_params = model.params.theta_vector()
est_params = result_ccpem_data['theta']
param_names = ['alpha_1 (intercept)', 'alpha_2 (mileage)', 'alpha_3 (type)', 'beta (discount)']

results_table = []
for i, name in enumerate(param_names):
    results_table.append([name, f"{true_params[i]:.4f}", f"{est_params[i]:.4f}"])
results_table.append(['pi (type prob)', f"{model.params.type_prob:.4f}", f"{result_ccpem_data['pi']:.4f}"])

print("\nStructural Parameter Estimates:")
print(tabulate(results_table, headers=["Parameter", "True", "Estimated"], tablefmt="simple"))

# Estimation statistics
stats_table = [
    ["Iterations", result_ccpem_data['n_iterations']],
    ["Computation time (s)", f"{result_ccpem_data['computation_time']:.2f}"],
]
print("\nEstimation Statistics:")
print(tabulate(stats_table, headers=["Metric", "Value"], tablefmt="simple"))



## 4.4 CCP-EM with Model Updating

Alternative to data updating: use the **model's CCP operator** to update CCPs.

### CCP-Model Approach (NPL-style)

In the M-step:

1. Given current CCPs $p^k$ and posterior weights $\tau^k$, estimate $\theta^{k+1}$

2. **Update CCPs using model operator**:
   $$p^{k+1} = \Psi(p^k; \theta^{k+1})$$
   
   where $\Psi$ maps CCPs to best-response CCPs given $\theta$.

This is similar to NPL (Aguirregabiria & Mira 2007) but with EM for unobserved heterogeneity.

In [ ]:
class CCPEMModelEstimator:
    """
    CCP-EM with Model updating + Initial Conditions correction.

    Hybrid approach matching MATLAB hetero10ns/shellbusdatacomp420cond.m:
    - FV from model CCPs solved to convergence (avoids first-stage logit bias)
    - Structural model likelihood (likeCCP) for type classification
    - Initial conditions: P(type=0|x₀,z) = Λ(γ'w)
    - β estimated as free coefficient + time dummies
    - Last period excluded
    """

    def __init__(self, data: pd.DataFrame, base_params: BusEngineParams = None):
        self.data = data.copy()
        self.base_params = base_params or BusEngineParams()

        self.bus_ids = sorted(data['bus_id'].unique())
        self.n_buses = len(self.bus_ids)
        self.T = data['period'].max() + 1
        self.bus_data = {}
        for bus_id in self.bus_ids:
            mask = data['bus_id'] == bus_id
            self.bus_data[bus_id] = {
                'x_idx': data.loc[mask, 'x_idx'].values,
                'z_idx': data.loc[mask, 'z_idx'].values[0],
                'replace': data.loc[mask, 'replace'].values,
                'periods': data.loc[mask, 'period'].values,
            }

        self._build_transitions()

        # Initial conditions data: [1, x_0, z]
        initial_data = data[data['period'] == 0].set_index('bus_id').loc[self.bus_ids]
        self.intcondX = np.column_stack([
            np.ones(self.n_buses),
            initial_data['x'].values,
            initial_data['z'].values,
        ])

        # Build stacked data arrays for structural estimation
        self._build_stacked_data()

    def _build_transitions(self):
        p = self.base_params
        self._P_continue = []
        self._P_replace = []
        for z in p.z_grid:
            P = build_transition_matrix(z, p.x_grid)
            self._P_continue.append(P)
            self._P_replace.append(P[0, :])

    def _build_stacked_data(self):
        """Build stacked arrays: data doubled, first half type=0, second half type=1."""
        p = self.base_params
        T, N = self.T, self.n_buses

        y_list, x_list, t_list, xidx_list, zidx_list = [], [], [], [], []
        for bus_id in self.bus_ids:
            bd = self.bus_data[bus_id]
            for t in range(T):
                y_list.append(bd['replace'][t])
                x_list.append(p.x_grid[bd['x_idx'][t]])
                t_list.append(t)
                xidx_list.append(bd['x_idx'][t])
                zidx_list.append(bd['z_idx'])

        y_s = np.array(y_list)
        self.y2 = np.concatenate([y_s, y_s])  # 2*N*T
        self.x_raw = np.concatenate([np.array(x_list)]*2)
        self.t2 = np.concatenate([np.array(t_list)]*2)
        self.xidx2 = np.concatenate([np.array(xidx_list)]*2)
        self.zidx2 = np.concatenate([np.array(zidx_list)]*2)
        self.s2 = np.concatenate([np.zeros(N*T), np.ones(N*T)])

        # Index: exclude last period
        self.index = self.t2 < (T - 1)

        # Structural regressors: [1, x_raw, s]
        self.xccp = np.column_stack([np.ones(2*N*T), self.x_raw, self.s2])

        # Time dummies for structural (T-2 dummies, for periods 1..T-2)
        self.td_struct = np.zeros((2*N*T, max(T-2, 0)))
        for t in range(1, T-1):
            self.td_struct[:, t-1] = (self.t2 == t).astype(float)

    # =========================================================================
    # CCP solving
    # =========================================================================

    def ccp_operator(self, ccps: np.ndarray, theta: np.ndarray, type_s: int) -> np.ndarray:
        """Single Bellman operator step: p' = Ψ(p; θ, s)"""
        params = BusEngineParams.from_vector(theta)
        p = self.base_params
        n_x, n_z = p.n_x, p.n_z
        ccps = ccps.reshape(n_z, n_x)
        ccps_new = np.zeros_like(ccps)
        u1 = params.intercept + params.mileage_coef * p.x_grid + params.type_coef * type_s
        for z_idx in range(n_z):
            p_replace = np.clip(1 - ccps[z_idx, :], 1e-6, 1 - 1e-6)
            V = EULER_CONSTANT - np.log(p_replace)
            v_diff = u1 + params.discount * (self._P_continue[z_idx] @ V - self._P_replace[z_idx] @ V)
            ccps_new[z_idx, :] = logit_prob(v_diff)
        return ccps_new.flatten()

    def solve_ccps(self, theta: np.ndarray, type_s: int,
                   initial: np.ndarray = None, tol: float = 1e-8, max_iter: int = 500) -> np.ndarray:
        """Solve for CCPs via fixed-point iteration."""
        p = self.base_params
        ccps = initial if initial is not None else 0.5 * np.ones(p.n_x * p.n_z)
        for it in range(max_iter):
            ccps_new = self.ccp_operator(ccps, theta, type_s)
            if np.max(np.abs(ccps_new - ccps)) < tol:
                return ccps_new
            ccps = ccps_new
        return ccps

    # =========================================================================
    # FV computation (model-based, without β)
    # =========================================================================

    def compute_fv(self, ccps0: np.ndarray, ccps1: np.ndarray) -> np.ndarray:
        """
        Compute FV for stacked data using model CCPs.

        FV(x,z,s) = [P(x'|x,z,keep) - P(x'|0,z,replace)] · [-log(P(replace|x',z,s))]

        Returns: fvt1 of length 2*N*T (stacked: type 0 then type 1)
        """
        p = self.base_params
        n_x, n_z = p.n_x, p.n_z
        N, T = self.n_buses, self.T
        ccps0_2d = ccps0.reshape(n_z, n_x)
        ccps1_2d = ccps1.reshape(n_z, n_x)

        FV_type = np.zeros((2, n_z, n_x))
        for s_idx, ccps_2d in enumerate([ccps0_2d, ccps1_2d]):
            for z_idx in range(n_z):
                V = -np.log(np.clip(1 - ccps_2d[z_idx, :], 1e-6, 1 - 1e-6))
                FV_type[s_idx, z_idx, :] = self._P_continue[z_idx] @ V - self._P_replace[z_idx] @ V

        # Build stacked FV array
        fvt1 = np.zeros(2 * N * T)
        for i, bus_id in enumerate(self.bus_ids):
            bd = self.bus_data[bus_id]
            z_idx = bd['z_idx']
            for t in range(T):
                x_idx = bd['x_idx'][t]
                fvt1[i * T + t] = FV_type[0, z_idx, x_idx]           # type 0 block
                fvt1[N * T + i * T + t] = FV_type[1, z_idx, x_idx]   # type 1 block
        return fvt1

    # =========================================================================
    # likeCCP: structural per-observation likelihood
    # =========================================================================

    @staticmethod
    def likeCCP(bccp: np.ndarray, y_keep: np.ndarray, X: np.ndarray) -> np.ndarray:
        """
        Per-observation likelihood P(y_obs | X, θ).

        P(keep|X) = σ(Xb). Returns P(y_observed | X).
        Matches MATLAB likeCCP.m: Like = (Y.*exp(U) + (1-Y)) ./ (1+exp(U))
        """
        sigma = logit_prob(X @ bccp)
        return y_keep * sigma + (1 - y_keep) * (1 - sigma)

    # =========================================================================
    # Initial Conditions
    # =========================================================================

    def intcond_nll(self, gamma: np.ndarray, base: np.ndarray) -> float:
        p0 = logit_prob(self.intcondX @ gamma)
        p = np.column_stack([p0, 1 - p0])
        marginal = np.sum(p * base, axis=1)
        return -np.sum(np.log(np.maximum(marginal, 1e-300))) / self.n_buses

    def intcond_posterior(self, gamma: np.ndarray, base: np.ndarray) -> np.ndarray:
        p0 = logit_prob(self.intcondX @ gamma)
        p = np.column_stack([p0, 1 - p0])
        joint = base * p
        marginal = np.sum(joint, axis=1, keepdims=True)
        return joint / np.maximum(marginal, 1e-300)

    # =========================================================================
    # EM Algorithm
    # =========================================================================

    def estimate(self, theta0: np.ndarray = None, max_iter: int = 100,
                 tol: float = 1e-7) -> dict:
        """
        Run CCP-EM with model updating + initial conditions.

        Algorithm:
        1. Solve CCPs to convergence with current θ
        2. Compute FV from model CCPs (without β)
        3. Compute type likelihoods via likeCCP (structural)
        4. Estimate initial conditions γ
        5. Compute posterior PType
        6. Re-estimate θ via weighted logit with PType weights
        """
        p = self.base_params
        N, T = self.n_buses, self.T

        if theta0 is None:
            theta0 = np.append(self.base_params.theta0_vector(), p.discount)

        theta = theta0.copy()
        ccps0 = 0.5 * np.ones(p.n_x * p.n_z)
        ccps1 = 0.5 * np.ones(p.n_x * p.n_z)
        gamma = np.zeros(3)
        gamma_start = np.zeros(3)

        idx = self.index  # exclude last period
        n_td = max(T - 2, 0)
        bccp = np.concatenate([theta[:4], np.zeros(n_td)])

        start_time = time.time()
        lp_history = []
        history = []

        for iteration in range(max_iter):
            # 1. Solve CCPs to convergence
            ccps0 = self.solve_ccps(theta, type_s=0, initial=ccps0)
            ccps1 = self.solve_ccps(theta, type_s=1, initial=ccps1)

            # 2. Compute FV from model CCPs
            fvt1 = self.compute_fv(ccps0, ccps1)

            # 3. Type likelihoods via structural model (likeCCP)
            X_struct = np.column_stack([self.xccp[idx], fvt1[idx], self.td_struct[idx]])
            y_keep = (1 - self.y2[idx]).astype(float)  # 1=keep
            Like = self.likeCCP(bccp, y_keep, X_struct)

            # Reshape: (N, T-1, 2) → bus-level likelihoods
            half = int(idx.sum()) // 2
            like0 = Like[:half].reshape(N, T-1)
            like1 = Like[half:].reshape(N, T-1)
            base0 = np.prod(np.clip(like0, 1e-300, None), axis=1)
            base1 = np.prod(np.clip(like1, 1e-300, None), axis=1)
            base = np.column_stack([base0, base1])

            # 4. Initial conditions (start after iteration 1)
            if iteration > 1:
                ic_start = gamma_start if iteration < 50 else gamma
                res_ic = minimize(self.intcond_nll, ic_start, args=(base,),
                                  method='BFGS', options={'disp': False})
                gamma = res_ic.x

            ll = self.intcond_nll(gamma, base)
            lp_history.append(ll)

            # 5. Posterior type probabilities
            PType = self.intcond_posterior(gamma, base)
            tau = PType[:, 1]
            pi_new = 1 - tau.mean()

            # 6. Expand PType to per-observation weights
            PType_obs = np.zeros(2 * N * T)
            for i in range(N):
                for t in range(T):
                    PType_obs[i * T + t] = PType[i, 0]
                    PType_obs[N * T + i * T + t] = PType[i, 1]

            # 7. Re-estimate structural parameters (weighted logit)
            X_struct_full = np.column_stack([self.xccp[idx], fvt1[idx], self.td_struct[idx]])
            PType_idx = PType_obs[idx]

            def wlogit_nll(b, Y, X, W):
                U = X @ b
                log1pexp = np.where(U > 20, U, np.log1p(np.exp(np.clip(U, -500, 20))))
                return np.sum(W * (log1pexp - Y * U))

            res_bccp = minimize(wlogit_nll, bccp, args=(y_keep, X_struct_full, PType_idx),
                              method='BFGS', options={'disp': False})
            bccp = res_bccp.x
            theta_new = bccp[:4]

            history.append({'theta': theta_new.copy(), 'pi': pi_new, 'll': ll, 'gamma': gamma.copy()})

            if iteration % 5 == 0:
                print(f"Iter {iteration+1}: θ={theta_new.round(3)}, "
                      f"π={pi_new:.3f}, γ={gamma.round(3)}, LL={ll:.4f}")

            # Convergence check
            if len(lp_history) > 26:
                c1 = abs((lp_history[-1] - lp_history[-26]) / max(abs(lp_history[-1]), 1e-300))
                c2 = abs((lp_history[-2] - lp_history[-27]) / max(abs(lp_history[-2]), 1e-300))
                if c1 < tol and c2 < tol:
                    print(f"Converged in {iteration+1} iterations")
                    break

            theta = theta_new

        return {
            'theta': theta_new,
            'pi': pi_new,
            'gamma': gamma,
            'tau': tau,
            'ccps0': ccps0,
            'ccps1': ccps1,
            'log_likelihood': ll,
            'n_iterations': iteration + 1,
            'computation_time': time.time() - start_time,
            'history': history
        }


In [ ]:
# Run CCP-EM with Model updating
print("="*70)
print("CCP-EM (Model Updating + Initial Conditions)")
print("="*70)

ccpem_model = CCPEMModelEstimator(data.drop(columns=['type_s']), model.params)
result_ccpem_model = ccpem_model.estimate()

# Compare posterior types to true types
pred_types_model = (result_ccpem_model['tau'] > 0.5).astype(int)
accuracy_model = (pred_types_model == true_types).mean()

# Display results
true_params = model.params.theta_vector()
est_params = result_ccpem_model['theta']
param_names = ['alpha_1 (intercept)', 'alpha_2 (mileage)', 'alpha_3 (type)', 'beta (discount)']

results_table = []
for i, name in enumerate(param_names):
    results_table.append([name, f"{true_params[i]:.4f}", f"{est_params[i]:.4f}"])
results_table.append(['pi (type prob)', f"{model.params.type_prob:.4f}", f"{result_ccpem_model['pi']:.4f}"])

print("\nStructural Parameter Estimates:")
print(tabulate(results_table, headers=["Parameter", "True", "Estimated"], tablefmt="simple"))

# Estimation statistics
stats_table = [
    ["Iterations", result_ccpem_model['n_iterations']],
    ["Computation time (s)", f"{result_ccpem_model['computation_time']:.2f}"],
]
print("\nEstimation Statistics:")
print(tabulate(stats_table, headers=["Metric", "Value"], tablefmt="simple"))


## 4.5 Summary: Methods Comparison

| Method | Handles Unobs. Het.? | Solves DP? | CCP Updates | Computational Cost |
|--------|---------------------|------------|-------------|-------------------|
| Two-Step CCP | **No** | No | N/A | Very Low |
| NFXP | Yes (with EM) | Yes (every θ) | N/A | Very High |
| CCP-EM (Data) | **Yes** | No | Weighted logit | Low |
| CCP-EM (Model) | **Yes** | No | Ψ(p; θ) | Low-Medium |

### Key Insights

1. **Two-Step fails** with unobserved heterogeneity because it cannot separate type-specific CCPs

2. **NFXP + EM** works but is computationally expensive (nested optimization within EM)

3. **CCP-EM (Data)**: Updates CCPs via weighted reduced-form estimation. Fast, but may be less efficient.

4. **CCP-EM (Model)**: Updates CCPs using structural model. Potentially more efficient, ensures CCPs are model-consistent.

In [ ]:
# Final comparison
print("="*70)
print("Summary: Estimation Results (With Observed Types)")
print("="*70)

# True parameters
true_params = model.params.theta_vector()
param_names = ['alpha_1 (intercept)', 'alpha_2 (mileage)', 'alpha_3 (type)', 'beta (discount)']

# Build comparison table for observed types
observed_table = []
for i, name in enumerate(param_names):
    observed_table.append([
        name,
        f"{true_params[i]:.4f}",
        f"{result_nfxp['theta'][i]:.4f}",
        f"{result_twostep['theta'][i]:.4f}"
    ])

print(tabulate(observed_table, headers=["Parameter", "True", "NFXP", "Two-Step"], tablefmt="simple"))

# Build comparison table for unobserved types
# Collect all results that exist
em_results = {}
for label, var in [('NFXP+EM', 'result_nfxp_em'), 
                    ('CCP-EM (Data)', 'result_ccpem_data'),
                    ('CCP-EM (Model)', 'result_ccpem_model')]:
    if var in dir():
        em_results[label] = eval(var)

headers = ["Parameter", "True"] + list(em_results.keys())
unobserved_table = []
for i, name in enumerate(param_names):
    row = [name, f"{true_params[i]:.4f}"]
    for res in em_results.values():
        row.append(f"{res['theta'][i]:.4f}")
    unobserved_table.append(row)

# Add pi row
pi_row = ["π (type prob)", f"{model.params.type_prob:.4f}"]
for res in em_results.values():
    pi_row.append(f"{res['pi']:.4f}")
unobserved_table.append(pi_row)

print("")
print("="*70)
print("Summary: Estimation Results (With Unobserved Types)")
print("="*70)
print(tabulate(unobserved_table, headers=headers, tablefmt="simple"))